# COGS 118B - Final Project

# Genre Fusion: Unsupervised Music Clustering

## Group members

- Brandon Kao
- Kyle Vu
- Connie Chang
- Catherine Zhang
- Grace Yang

# Abstract 
This project aims to investigate the complexities of music genres and features that present in songs. A song can overlap between many different genres, making it difficult to formally classify it. The dataset used contains audio features associated with Spotify tracks. We selected tracks from the 10 most popular and 10 least popular genres to get a wide range of song selection. After cleaning the data, we ran several different clustering models to create visualizations. Our results show that certain features had a greater impact on clustering than others. We measured this by finding the mean and median values of each feature for each of the clusters. 

# Background

Over the past 20 years, the development of electronic methods for music distribution and streaming services has decreased the barrier of entry into the music industry, giving rise to new genres and subcultures[<sup>[1]</sup>](#lorenznote). To account for this growth, extensive research has gone into automating genre classification using AI. In 2002, a study published in IEEE Transactions on Speech and Audio Processing trained a model on timbral texture, rhythmic content and pitch content of various songs[<sup>[2]</sup>](#lorenznote). When tasked with labeling songs with their genres, the model achieved an accuracy of 61%, which was comparable to the accuracy of humans[<sup>[2]</sup>](#lorenznote). Since then, models for music genre identification have grown, and are used by music streaming services to recommend tracks to their users. On Free Music Archive, a streaming service for royalty free music, a majority of their modes for genre identification employ Support Vector Classifiers(SVD), Logistic Regression(RNN), and Convolutional Neural Networks(CNN)[<sup>[3]</sup>](#lorenznote). Traditionally, genre classification has been approached in research with supervised learning models, which positions the label as an innate trait of a song.

However, treating genre classification as a supervised learning problem fails to account for the evolving nature of music and art. Categorization can be difficult when genre definitions often vary from person to person or from their technical definitions[<sup>[2]</sup>](#lorenznote). The defining songs or traits of a genre can be hazy or ever-changing, and “problems occur if we take in account music genre categories as stable and essential entities”[<sup>[2]</sup>](#lorenznote). One example of how genre labels change is the removal of the broad label Latin from Spotify's genre classifier in 2022 in favor of more precise categories like Latin Pop, Urbano Latino, and Latin Hip Hop[<sup>[4]</sup>](#lorenznote). Some music genre identifier models have approached the problem using unsupervised learning. Spotify’s recommender system uses a song's audio data and its location in playlists to identity new genres, or create unique groupings that can not be described by an existing genre label[<sup>[5]</sup>](#lorenznote). Such methods attempt to accommodate how music labels are evolving and transient in nature. 


# Problem Statement

A challenge within the music industry is the ambiguity surrounding the classification of music genres.  Despite numerous attempts to delineate genre boundaries, there remains a notable absence of consensus on the defining characteristics of each genre. For instance, a single song may straddle multiple genres, such as pop and hip-hop, blurring traditional distinctions. This lack of clarity hinders effective communication and analysis within the field. A song contains many features such as, energy, loudness, tempo, speechiness, popularity, etc that can all be numerically quantified. However, alone, it is unclear what the correlations between each of those features are in regard to genre. Thus, we aim to create a model that clusters similar songs to see what features contribute to song similarity and bring distinct sound to a genre.

# Data

[Dataset Link](https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset)

The dataset “🎹 Spotify Tracks Dataset” by user Maharshi Pandya on Kaggle compiles 114,000 tracks, or observations, from Spotify. Each track is measured on over 20 variables, which detail the track’s name, artist(s), audio features, sentiment, and more. The variable “track_genre” is the music genre the track belongs to. Since each observation can only be labeled by one genre, the dataset contains multiple entries of each track with duplicate information besides a different genre.The “popularity” column scores how trending a song is from 0 to 100, with 100 being the highest. The popularity is calculated by the amount of plays, with more recent plays being weighted higher. Other important variables track the qualities of each song on a scale from 0.0 to 1.0, such as “danceability” and “valence” to measure the danceability and positive mood of the song. 

We decided to limit our dataset to songs only belonging to the top 10 most popular genres and top 10 least popular genres (20 total). This was done in order to reduce the complexity of our dataset, as attempting to cluster around 114 unique genres would lead to high computational complexity and results that would be difficult to interpret. By selecting genres with a wide range of popularity, we are still able to get a varied representation of songs.

For our data cleaning process, we took typical steps in order to get our data in the form we wanted. We started by removing all fully duplicate rows as well as rows with missing values. To input the data into the model, we also limited the dataset to the musical characteristic features, removing columns that were related to identification, like track_id or the album the song belonged to. We also excluded certain redundant/not useful features like whether or not a song was explicit rather than include it via a one-hot encoding.


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [2]:
# Load data
pd.set_option('display.max_columns', None)  

df = pd.read_csv('data/dataset.csv')

There are too many generes in our dataset, we're only interested in the most popular generes, so we looked for the top 20 genres with the highest average popularity 

In [3]:
music_features = df[['popularity', 'duration_ms', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'track_genre']]
mean_features = music_features.groupby('track_genre').mean()

# sort by popularity and display results
for genre in mean_features.sort_values('popularity', ascending=False).index[0:20]: #ascending=False for most popular
    print(genre)
df = df[(df['track_genre'].isin(mean_features.sort_values('popularity', ascending=True).index[0:10])) | (df['track_genre'].isin(mean_features.sort_values('popularity', ascending=False).index[0:10]))]


pop-film
k-pop
chill
sad
grunge
indian
anime
emo
sertanejo
pop
progressive-house
piano
mandopop
deep-house
brazil
electronic
pagode
ambient
british
metal


For the following features, we want to:

- popularity, scale it
- duration_ms, scale it
- explicit, one hot encode
- danceability, scale it
- energy, scale it
- key, one hot encode
- loudness, scale it
- mode, one hot encode
- speechiness, scale it
- acousticness, scale it
- intrumentalness: scale it
- liveness: scale it
- tempo: scale it
- time_signature: scale it

In [6]:
df2 = df.copy()

df2 = df2.drop_duplicates(subset='track_id').dropna()
df0 = df2.copy()

# Define columns to scale (excluding 'explicit', 'key', and 'mode' for now)
scale_columns = ['popularity', 'duration_ms', 'danceability', 'energy', 'loudness', 
                 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'tempo']

# Define the scaler
scaler = MinMaxScaler()

# Apply scaling on the selected columns
df2[scale_columns] = scaler.fit_transform(df2[scale_columns])
df2.head()

# One-hot encode 'key' and 'mode'
df2 = pd.get_dummies(df2, columns=['key', 'mode', 'explicit', 'time_signature'], drop_first=True)
df2 = df2.drop(['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'track_genre'], axis=1)


# Proposed Solution

Judging a song’s qualities solely based on its genre label can be inaccurate, since genre labels can be broad, their definitions can be inconsistent, and many songs can be classified into multiple genres. To create groupings between songs that are based on true similarities, we will use unsupervised learning to cluster ~20,000 music tracks based on their attribute scores like popularity, energy, instrumentalness, and tempo. First, we will dimensionally reduce the data using PCA or UMAP, and use feature scaling to ensure no variable is skewing the data. We will then compare the effectiveness of KMeans and Spectral Clustering by contrasting the different clusters that emerge and the overall trends we can evaluate from music. Doing so will uncover novel groupings between different tracks that are more tailored to the songs themselves. 

# Evaluation Metrics

One evaluation metric we can use is to get the silhouette scores of our formed clusters. This gives us insight into how well-defined each of our clusters are. Silhouette scores range from -1 to 1, and higher scores indicate that the clusters are more distinct from each other.

We can calculate these scores for each point by using the standard formula, $\frac{b-a}{max(a,b)}$, where $a$ is the mean distance between a point and all other points in its cluster, and $b$ is the mean distance between a point and all data points in the nearest other cluster. From this formula, we can see that silhouette score improves when $b$ is a larger value (clusters are more spread apart) and $a$ is a smaller value (points within a cluster are close together). 

By comparing the silhouette scores we get from the various clustering techniques we try, we will be able to determine which techniques form the better clusters, thus giving a more meaningful result. 


# Results

![image](elbow.png)
![image](umap_kmm.png)
*Further code/exploration can be found in the file PCA_and_UMAP_K_Means_Clustering.ipynb*

From the inertia plot above, we decided to use 3 clusters for the K-means clustering with UMAP dimensionality reduction. There were distinct clusters being formed via this method, and each cluster formed is of a similar size. After calculating a silhouette score for this plot, we got a score of 0.56, indicating a fairly good distinction between clusters. This method was successfully able to find distinct features between the 3 clusters.

Similarly, we decided to choose to use 2 clusters when doing K-means clustering with PCA reduction. By using 2 clusters, we have a very clear distinction between the two clusters. As such, we ended up with a high silhouette score of 0.62. This makes sense, because using smaller numbers of clusters generally leads to higher silhouette scores. Although this method was able to form distinct clusters, using only 2 of them might make the result not as meaningful. 

![image](pca_spec.png)
![image](spec_umap.png)
*Further code/exploration can be found in the file PCA&UMAP_Spectral_Clustering.ipynb*

The results of our spectral clustering was very similar to that of the simple K-means plots. This may have been because we used the same preprocessing and dimensionality reduction techniques to generate these plots, leading to more separated clusters that would be handled similarly by both techniques. Ultimately, these plots did not add much more information or offer new perspectives on the data. 

![image](genre.png)

Above are 3 tables representing the distribution of different genres across the 3 main clusters we formed.

We decided to focus our analysis on the clustering produced by the UMAP and K-means approach. This clustering had a fairly high silhouette score while also having a reasonable number of clusters, so we felt it would be most meaningful to interpret the results from that plot. 
Cluster 0
Many of the songs in the top four genres are dominated by songs outside of North America, such as kpop and pop-film, which is dominated by soundtracks from popular Indian films. Songs in Cluster 0 scored high in energy, were generally louder, and had more danceability than songs in the other clusters, though not by a significant margin. The main standout feature of Cluster 0 was its popularity; songs in this cluster had a much higher mean and median popularity compared to songs in other clusters (a mean of 53.77 versus 16.48 and 5.23 for Clusters 1 and 2 respectively). When we look at the genres in this cluster, we can see it is composed of both mainstream music genres as well as smaller genres with more dedicated fanbases, which might explain the high popularity. 

The high percentages of songs belonging to Cluster 0 is also explained by the fact that this cluster contained noticeably more songs than Clusters 1 and 2. While Clusters 1 and 2 had around 5000 tracks each, cluster 0 contained closer to 8000 tracks. 
Cluster 1
With genres like detroit-techno, IDM, and grindcore, Cluster 1 demonstrated strong representation from electronic music. Songs in Cluster 1 scored high in instrumentalness, meaning that the tracks represented by this cluster commonly contained no or few lyrics. Specifically, the average instrumentalness in this cluster was 0.8 compared to 0.01 and 0.02 from Clusters 0 and 2. This explains a thread between some of the genres in this cluster that seem to be less related, like classical music versus grindcore. Another notable feature in this cluster was its lower valence, meaning tracks in this cluster tended to sound more negative (e.g. sad, angry, etc.) Representation from genres like IDM, Iranian, and Grindcore likely contributed strongly towards this.
Cluster 2
Cluster 2 was largely defined by its popularity measure, including romance, latin, and kids music. Its low popularity suggests that these genres are the least played on Spotify. Songs in Cluster 2 did not have notable scores in any particular feature besides popularity. Instead, it usually remains as a middle point between features of Cluster 0 and Cluster 1. The only feature which it had a higher mean for was acousticness. Cluster 2 had a mean of 0.4 (compared means of 0.34 and 0.35 for Cluster 0 and Cluster 1 respectively). This implies that without a defining feature, these genres are not often sought out. 




# Discussion

### Interpreting the result

popularity, loudness, acousticness, valence, instrumentalness, energy


From these results, we are able to observe features that can define a music genre and features that do not have much of an impact on genre.

One striking feature was how much instrumentalness played a part in the clustering of our groups. This may indicate that having a song focus on instrumentals is a more unusual and rare choice, and many artists prefer having lyrics in their songs. Many of the songs in cluster 1 were part of the EDM and techno genre, which indicates how distinct this modern genre is from other forms of music. Considering that these genres were not super present in Cluster 0, which contained the most popular genres, this may indicate that electronic music is not as popular. The fact that many of the songs from Cluster 0 were from genres in Latin America and Asia indicates that music from these countries will become more dominant in American music charts in the future. This indicates music tastes are becoming more globalized over time, as users are exposed to songs from different countries than their own through the internet.

Cluster 2 has overall less popularity, yet many of the genre titles in Cluster 2 are still recognizable. This may be because many of the songs that we consider part of these genres could have been classified under other genre titles instead. With the wide amount of possible names for genres, this indicates how genres overlap and their definitions can vary based on opinion.

Features that did not play a large role in the clustering were key, explicitness, and mode. They had consistent means across all three clusters. This means that regardless of genre, these tracks were not defined by the key they were in, the presence of explicit lyrics, and the modality.


### Limitations

We faced limitations in computational power, especially when we attempted to use the GridSearch function with UMAP and Kmeans. Due to the size of our dataset, we wanted to try UMAP with a n_neighbors value of at least over 1000, but our devices did not have enough power to handle that many comparisons. Although the dataset we chose was detailed and contained 20 variables in features of music, there was no parameter for sentiment analysis of the lyrics. The dataset had a variable detailing the explicitness, as well as the amount of words in the song, but the exact lyrics of the song were not considered a major part of model’s analysis. In many genres, lyrics are a part of the artistry and meaning behind a track. Including this feature of the musical experience could lead to new correlations between different songs and genres. 

Additionally, our model is limited by the scope of the dataset we chose. The inclusion of more data from different genres than just the 10 most popular or 10 least popular tracks could have provided new insights. Basing our genre choice on Spotify’s popularity score algorithm can also be an inaccurate way to obtain a wide range of songs, depending on how their algorithm functions. It could have also biased our models unintentionally into clustering based on popularity. However, evaluating more than 20 genres in the model could cause more problems in computational power and running SciKit learn.


### Ethics & Privacy

When AI models are trained on creative art data, ethical concerns naturally arise over whether the artist consented to their work being used. Modern artists often face issues with their works getting used or pirated without proper payment or crediting. We note our model is trained on publicly sourced data from Spotify, where artists agree to license and distribute their tracks to be analyzed by Spotify’s API. We credit the person who compiled the original dataset as well as the artists who produced the songs in the dataset in order to mitigate concerns in copyright. 
Building a comprehensive model for music pattern analysis requires using a diverse dataset of genres and music styles to avoid our model being biased. Due to their popularity, pop or hip-hop tracks can be overrepresented, leading to the marginalization of niche or emerging genres. Furthermore, datasets that primarily focus on trending music in the United States can exclude artists from non-English speaking countries and non-Western music styles. 
To mitigate biases, we choose a dataset that contains over 100 different genres, with each genre having an equal amount of 1,000 observations. This prevents an overrepresentation of specific artists or genres to make sure our model fairly represents the type of music styles we plan to showcase. Although Spotify has a large base of users in North America, around 50% of streams from Spotify are from Asia and Latin America[<sup>[4]</sup>](#lorenznote). Since the popularity score, which determined the models we studied, is based on streaming, our model does not solely focus on Western music styles. This helps mitigate biases in underrepresenting marginalized groups in research.

### Conclusion

In conclusion, features like popularity and instrumentalness help define certain genres of music while others like key and mode do not. This fits into the production of new music, suggesting to musicians what kinds of music may be more popular and perform better. It can also suggest ways to evolve genres and experiment with features not commonly found in a particular music area. However, this categorization of music by genre does have its limitations and should be further explored.

In the future, we can continue to expand on this by including a larger range of genres and features. Additionally, now that we know most genres of music generally have similar features, we can move forward with the idea of building a model that generates music. This was our initial project idea, but was hindered by the lack of easily accessible data and initial questions surrounding music genres. 


# Footnotes

1.^: Wikström, Patrik. “The Music Industry In an Age of Digital Distribution.” OpenMindBBVA, OpenMindBBVA, www.bbvaopenmind.com/en/articles/the-music-industry-in-an-age-of-digital-distribution/. Accessed 20 Mar. 2024. 

2.^: Cerati, Gabriele. (2021). Difficult to define, easy to understand: the use of genre categories while talking about music. SN Social Sciences. 1. 10.1007/s43545-021-00296-2. 

3.^: George Tzanetakis and Perry Cook. Musical genre classification of audio signals. Speech and Audio Processing, IEEE transactions on, 10(5):293–302, 2002.

4.^: Daniels, Matt. “You Should Look at This Chart about Music Genres.” The Pudding, The Pudding , pudding.cool/2023/10/genre/. Accessed 20 Mar. 2024. 

5.^: Pastukhov, Dmitry. “How Spotify’s Algorithm Works? A Complete Guide to Spotify Recommendation System [2022]: Music Tomorrow Blog.” How Spotify’s Algorithm Works? A Complete Guide to Spotify Recommendation Algorithms, Music Tomorrow Blog, 9 Feb. 2022, www.music-tomorrow.com/blog/how-spotify-recommendation-system-works-a-complete-guide-2022. 
